In [1]:
# coding: utf-8
import os

import pandas as pd
import numpy as np
import scipy
import lightgbm as lgb
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns

from mydatools.plot import plot_multiclass_feature_dist
from mydatools.features_analyze import get_top_k_corr

% matplotlib inline

## Config

In [2]:
trn_path = './data/input/application_train.csv'
tst_path = './data/input/application_test.csv'
id_col = 'SK_ID_CURR'
label_col = 'TARGET'

submission_path = './data/output/submission/submission.csv'
output_id_col = id_col
output_label_col = label_col

## Load Data

In [3]:
trn_df = pd.read_csv(trn_path)
trn_df['ds_type'] = 'trn'
trn_df[label_col] = trn_df[label_col].astype('int')

tst_df = pd.read_csv(tst_path)
tst_df['ds_type'] = 'tst'

full_df = pd.concat([trn_df, tst_df])

del(trn_df)
del(tst_df)

full_df.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,TOTALAREA_MODE,WALLSMATERIAL_MODE,WEEKDAY_APPR_PROCESS_START,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE,ds_type
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0149,"Stone, brick",WEDNESDAY,0.9722,0.9722,0.9722,0.6192,0.6243,0.6341,trn
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0714,Block,MONDAY,0.9851,0.9851,0.9851,0.7960,0.7987,0.8040,trn
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,MONDAY,NaN,NaN,NaN,NaN,NaN,NaN,trn
3,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,WEDNESDAY,NaN,NaN,NaN,NaN,NaN,NaN,trn
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,THURSDAY,NaN,NaN,NaN,NaN,NaN,NaN,trn


In [4]:
# dataset type
is_trn = full_df['ds_type'] == 'trn'
is_tst = full_df['ds_type'] == 'tst'

## Features

In [5]:
feature_columns = []

def add_features(features):
    if not isinstance(features, list):
        features = [features]
    global feature_columns
    feature_columns.extend([f for f in features if f not in feature_columns])

**numerical_features**

In [6]:
numerical_features = full_df.dtypes[full_df.dtypes != 'object'].index.tolist()
numerical_features = [c for c in numerical_features if c not in [id_col, label_col, 'ds_type']]
add_features(numerical_features)

**categorical_features**

In [7]:
categorical_features = full_df.dtypes[full_df.dtypes == 'object'].index.tolist()
full_df[categorical_features].nunique()

CODE_GENDER                    3
EMERGENCYSTATE_MODE            2
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
NAME_CONTRACT_TYPE             2
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
NAME_INCOME_TYPE               8
NAME_TYPE_SUITE                7
OCCUPATION_TYPE               18
ORGANIZATION_TYPE             58
WALLSMATERIAL_MODE             7
WEEKDAY_APPR_PROCESS_START     7
ds_type                        2
dtype: int64

In [8]:
categorical_features = full_df.dtypes[full_df.dtypes == 'object'].index.tolist()
categorical_features = categorical_features[:-1] # 为了去掉ds_type
# 去掉分类数太多
categorical_features = full_df[categorical_features].columns[full_df[categorical_features].nunique() <= 10]

# get dummies
full_df = pd.get_dummies(full_df, columns=categorical_features, prefix_sep=':')

# add features
new_features = full_df.columns.tolist()[full_df.columns.tolist().index('ds_type') + 1:]
add_features(new_features)

**bureau.csv**

In [9]:
bureau = pd.read_csv('./data/input/bureau.csv')

bureau = pd.get_dummies(bureau, columns=bureau.columns[bureau.dtypes == 'object'].tolist())
bureau = bureau.fillna(0)

bureau = bureau.groupby('SK_ID_CURR').mean()

bureau.columns = ['bureau_'+f for f in bureau.columns.tolist()]

In [10]:
full_df = full_df.join(bureau, on='SK_ID_CURR')
add_features(bureau.columns.tolist())

## LightGBM

In [11]:
trn_df = full_df[is_trn]
tst_df = full_df[is_tst]

X = trn_df[feature_columns]
y = trn_df[label_col]
X_trn, X_val, y_trn, y_val = train_test_split(X, y, test_size=0.20, random_state=20180521)

X_tst = tst_df[feature_columns]

d_trn = lgb.Dataset(X_trn.values, y_trn)
d_val = lgb.Dataset(X_val.values, y_val)

In [12]:
params = {
    'application': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'max_depth': 4,
    'num_leaves': 20,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.5,
    'bagging_freq': 1,
    'is_unbalance': True,
    'seed': 0,
    'reg_alpha': 0.1,
}
bst = lgb.train(params, d_trn, 500, valid_sets=[d_trn, d_val], early_stopping_rounds=50)

[1]	training's auc: 0.702732	valid_1's auc: 0.704609
Training until validation scores don't improve for 50 rounds.
[2]	training's auc: 0.708955	valid_1's auc: 0.711792
[3]	training's auc: 0.716048	valid_1's auc: 0.718881
[4]	training's auc: 0.718791	valid_1's auc: 0.722166
[5]	training's auc: 0.719147	valid_1's auc: 0.723237
[6]	training's auc: 0.719951	valid_1's auc: 0.724779
[7]	training's auc: 0.721018	valid_1's auc: 0.725692
[8]	training's auc: 0.722769	valid_1's auc: 0.727256
[9]	training's auc: 0.722853	valid_1's auc: 0.726938
[10]	training's auc: 0.72347	valid_1's auc: 0.727674
[11]	training's auc: 0.725199	valid_1's auc: 0.729866
[12]	training's auc: 0.726151	valid_1's auc: 0.730504
[13]	training's auc: 0.727934	valid_1's auc: 0.732763
[14]	training's auc: 0.728943	valid_1's auc: 0.733916
[15]	training's auc: 0.72906	valid_1's auc: 0.734154
[16]	training's auc: 0.72899	valid_1's auc: 0.734089
[17]	training's auc: 0.729128	valid_1's auc: 0.734264
[18]	training's auc: 0.729241	va

[152]	training's auc: 0.770017	valid_1's auc: 0.76419
[153]	training's auc: 0.770046	valid_1's auc: 0.764219
[154]	training's auc: 0.770209	valid_1's auc: 0.764357
[155]	training's auc: 0.770275	valid_1's auc: 0.764371
[156]	training's auc: 0.770388	valid_1's auc: 0.764457
[157]	training's auc: 0.770483	valid_1's auc: 0.764495
[158]	training's auc: 0.770618	valid_1's auc: 0.764587
[159]	training's auc: 0.770662	valid_1's auc: 0.76459
[160]	training's auc: 0.770793	valid_1's auc: 0.764629
[161]	training's auc: 0.770992	valid_1's auc: 0.764649
[162]	training's auc: 0.771094	valid_1's auc: 0.764714
[163]	training's auc: 0.771216	valid_1's auc: 0.764756
[164]	training's auc: 0.771341	valid_1's auc: 0.764766
[165]	training's auc: 0.771417	valid_1's auc: 0.764817
[166]	training's auc: 0.771502	valid_1's auc: 0.764825
[167]	training's auc: 0.771594	valid_1's auc: 0.76476
[168]	training's auc: 0.771772	valid_1's auc: 0.764892
[169]	training's auc: 0.771854	valid_1's auc: 0.764893
[170]	trainin

[303]	training's auc: 0.785034	valid_1's auc: 0.767704
[304]	training's auc: 0.785096	valid_1's auc: 0.767714
[305]	training's auc: 0.785191	valid_1's auc: 0.767638
[306]	training's auc: 0.78526	valid_1's auc: 0.767631
[307]	training's auc: 0.785324	valid_1's auc: 0.767681
[308]	training's auc: 0.785424	valid_1's auc: 0.767684
[309]	training's auc: 0.78555	valid_1's auc: 0.76769
[310]	training's auc: 0.785644	valid_1's auc: 0.767716
[311]	training's auc: 0.78574	valid_1's auc: 0.76772
[312]	training's auc: 0.785894	valid_1's auc: 0.767749
[313]	training's auc: 0.786006	valid_1's auc: 0.767783
[314]	training's auc: 0.786068	valid_1's auc: 0.767804
[315]	training's auc: 0.786138	valid_1's auc: 0.767791
[316]	training's auc: 0.786195	valid_1's auc: 0.767801
[317]	training's auc: 0.786295	valid_1's auc: 0.7678
[318]	training's auc: 0.786342	valid_1's auc: 0.767727
[319]	training's auc: 0.786381	valid_1's auc: 0.767748
[320]	training's auc: 0.786471	valid_1's auc: 0.767737
[321]	training's 

[455]	training's auc: 0.796732	valid_1's auc: 0.767981
[456]	training's auc: 0.796769	valid_1's auc: 0.767972
[457]	training's auc: 0.796811	valid_1's auc: 0.76797
[458]	training's auc: 0.796856	valid_1's auc: 0.767952
[459]	training's auc: 0.796883	valid_1's auc: 0.767942
[460]	training's auc: 0.796954	valid_1's auc: 0.767945
[461]	training's auc: 0.796997	valid_1's auc: 0.767909
[462]	training's auc: 0.797037	valid_1's auc: 0.76793
Early stopping, best iteration is:
[412]	training's auc: 0.793899	valid_1's auc: 0.768133


In [13]:
# bst = lgb.train(params, trn_lgb, 1000, valid_sets=[trn_lgb, val_lgb], early_stopping_rounds=20,
#                 init_model=bst, 
#                 learning_rates=lambda iter: 0.1 * (0.99 ** iter))

In [14]:
imp_df = pd.DataFrame([bst.feature_importance()], columns=feature_columns, index=['importance']).T.sort_values(by='importance', ascending=False)
imp_df

,importance
EXT_SOURCE_3,368
EXT_SOURCE_2,351
EXT_SOURCE_1,321
DAYS_BIRTH,269
AMT_CREDIT,208
AMT_GOODS_PRICE,202
AMT_ANNUITY,187
DAYS_EMPLOYED,164
bureau_AMT_CREDIT_SUM,155
DAYS_REGISTRATION,139


## predict

In [15]:
res_df = pd.DataFrame(bst.predict(X_tst), columns=[output_label_col])
res_df[output_id_col] = tst_df[output_id_col].values
res_df[[output_id_col, output_label_col]].to_csv(submission_path, index=False)